### Generating the MnAu2 cell structure inputs for the QE calculations

Read in an initial .cif file (6-atoms) and expand to 54 atoms cell by translating in the x,y directions to increase the number of Mn atoms. This allows for more magnetic patterns in the Mn layers. Save to file as a .cif. 
The patterns matrix and coordinateis saved to file are used to generate the input for calculating configuation energies.
Total configuration energies and the pattern matrix are input for the exchange model to calculate the exchange parameters. 


In [223]:
from pathlib import Path
from sys import stdout
import numpy as np
import pandas as pd
import itertools
from pymatgen import units
from pymatgen.transformations.standard_transformations import SupercellTransformation
import sympy
from scipy.constants import Boltzmann, electron_volt
from sympy import symbols
from sympy.vector import CoordSys3D

import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression


from ruamel.yaml import YAML
yaml = YAML()

from neighbormodels.structure import from_file
from neighbormodels.neighbors import count_neighbors
from neighbormodels.interactions import build_model

kB = 1000 * Boltzmann / electron_volt

pd.set_option("display.colheader_justify", "left")
pd.set_option("display.html.border", 0)
html_table_style = {"selector": "th", "props": [("text-align", "left")]}

### Starting with 6 atom cell:

In [252]:
cif_filepath = "data/mnau2-6.cif"
structure_mnau26 = from_file(structure_file=cif_filepath)
print(structure_mnau26)

Full Formula (Mn2 Au4)
Reduced Formula: MnAu2
abc   :   3.394839   3.394839   8.880636
angles:  90.000000  90.000000  90.000000
Sites (6)
  #  SP      a    b         c
---  ----  ---  ---  --------
  0  Mn    0    0    0
  1  Au    0.5  0.5  0.840319
  2  Au    0.5  0.5  0.159681
  3  Mn    0.5  0.5  0.5
  4  Au    0    0    0.340319
  5  Au    0    0    0.659681


### Translated to create the 54 Atom cell: 

In [254]:
structure_mnau254 = structure_mnau26.copy()
structure_mnau254.make_supercell([3,3,1])
structure_mnau254.to(filename="data/mnau2-54.cif")
#print(structure_mnau254)

### Generate magentic pattern permutations for the 18 Mn atoms in the 54 atom cell: 

In [236]:
#512 possible permutaions for 9 atoms (single layer Mn) and either 1,-1 spin value
patterns1 = list(itertools.product([1,-1], repeat=9))
pmatrix = np.zeros((len(patterns1), 9))
for i in range(len(patterns1)):
    pmatrix[i] = np.array(patterns1[i])
p_matrix = pmatrix.transpose()

#Use matrix of Mn permutations together with Au matrix (spin 0) layered to create whole table of patterns
#taking only the first 200 cases here.
Mn_layer = pd.DataFrame(p_matrix[:,0:200])
Au_layer = pd.DataFrame(np.zeros((18,200)))
pattern_matrix_g = pd.concat([Mn_layer,Mn_layer*-1,Au_layer,Au_layer], ignore_index=True)

#Create part of QE input file by mapping the sites to either Mn or Au and writing to file.
#Additional script and modified coordinates from mnau2.cif generates the actual input files.
map_Mn = {1: "Mn1", -1:"Mn2", 0: "Au"}
for i in range(200):
    pattern_matrix_g[i] = pattern_matrix_g[i].map(map_Mn)

pattern_matrix_g.to_csv("data/mg_patterns-g.list", header=False, index=False, sep='\t')

### Getting the neighbor information to calculate the exchange parameters:

Rearrange the pattern matrices to match the atomic sequence in the structure and create pattern matrix useable 
in the exchange model calculation.  


In [180]:
nm_col = pd.concat([Mn_layer[0]*0,Au_layer[0],Mn_layer[0]*0,Au_layer[0]],ignore_index=True)
fm_col = pd.concat([Mn_layer[0],Au_layer[0],Mn_layer[0],Au_layer[0]], ignore_index=True)
pattern_matrix_g = pd.concat([Mn_layer,Au_layer,Mn_layer*-1, Au_layer],  ignore_index=True)
pattern_matrix = pd.concat([nm_col, fm_col, pattern_matrix_g],  axis=1)


types =[]
types.append("nm")
types.append("fm")
index = np.arange(1,len(pattern_matrix.columns)-1)
for i in index:
    types.append("g-"+str(i))
    
pattern_matrix.columns = types
xyz = pd.concat([pd.DataFrame(structure_mnau254.species,columns=["species"]), 
                 pd.DataFrame(structure_mnau254.cart_coords, columns=["x","y","z"])],
                              axis=1)

mnau2_patterns = pd.DataFrame( pd.concat([ xyz,pattern_matrix], axis=1 ) )

#save to file for visualization.
mnau2_patterns.to_csv("data/mnau2_patterns.xyz", index=False, sep='\t')




### Calculate the neighbor energies from the magnetic patterns:

In [234]:
mod_structure_mnau2 = structure_mnau254.copy()
mnau2_neighbor_data = count_neighbors(cell_structure=mod_structure_mnau2, r=6.8)
exchange_model = build_model(magnetic_patterns=pattern_matrix, neighbor_data=mnau2_neighbor_data)

#drop the Au atoms
exchange_model = exchange_model[exchange_model.columns.drop(list(exchange_model.filter(regex='Au')))]

#exchange_model

### Calculated energies for different patterns using QE:

In [274]:
#Calculated energies from QE for the different patterns are tabulated and read in
qe_energies = pd.read_csv('data/energy.out.200', header=None)
qe_energies.columns = ["pattern", "num_sites", "total_energy" ]

#qe_energies

Calculated energies and values from neighbor model matched according to pattern type. 
The enegies are normalized relative to the "nm" type (spin 0).

In [275]:
model_matrix = qe_energies \
    .assign(energy = lambda x:
        (x["total_energy"] -
         np.float64(x.query("pattern == 'nm'").loc[:, "total_energy"])) *
        1000 * units.Ha_to_eV / x["num_sites"])  \
    .merge(exchange_model, on=["pattern"]) \
    .query("pattern != 'nm'") \
    .assign(J1 = lambda x: x["J1_MnMn"]) \
    .assign(J2 = lambda x: x["J2_MnMn"]) \
    .assign(J3 = lambda x: x["J3_MnMn"]) \
    .assign(J4 = lambda x: x["J4_MnMn"]) \
    .loc[:, ["pattern", "energy", "J1", "J2", "J3", "J4"]]


#model_matrix

### Fitting to get final parameters:

Final matrix of energies and exchange model is used in a fit function to get the values of the J parameters.
2 results are compared, and values are in units of Rydberg.

In [276]:
smf_exchange_fit = smf.ols(data=model_matrix, formula="energy ~ J1 + J2 + J3 + J4").fit()
smf_exchange_parameters = pd.DataFrame(smf_exchange_fit.params, columns=["stats_models"])
#smf_exchange_parameters

In [277]:
lm = LinearRegression()
lm_exchange_fit = lm.fit(X=model_matrix[["J1", "J2", "J3", "J4"]], y=model_matrix["energy"])
lm_exchange_parameters = pd.DataFrame({
    "sklearn": [lm_exchange_fit.intercept_, 
                lm_exchange_fit.coef_[0], lm_exchange_fit.coef_[1],lm_exchange_fit.coef_[2],lm_exchange_fit.coef_[3]]},
    index=["Intercept", "J1", "J2", "J3", "J4"],
)
#lm_exchange_parameters

In [278]:
exchange_parameters = pd.concat([smf_exchange_parameters,lm_exchange_parameters], axis=1)
#save to file    
exchange_parameters.to_csv("data/mnau2_exchange_parameters.200.txt", sep=' ')
    

exchange_parameters

,stats_models,sklearn
Intercept,-3951.055077,-3951.055077
J1,-20.935496,-20.935496
J2,-28.385814,-28.385814
J3,7.135843,7.135843
J4,-20.935496,-20.935496
